# Response Times

In [13]:
# Import Python standard library and IPython packages we need.
import os
import subprocess
import sys
import csv
import numpy as np
import matplotlib.pyplot as plt
from collections import defaultdict
from pprint import pprint
import json
import time
import requests
import statistics
from requests.auth import HTTPBasicAuth

# Ask GRASS GIS where its Python packages are.
gisbase = subprocess.check_output(["grass", "--config", "path"], text=True).strip()
os.environ["GISBASE"] = gisbase
os.environ["ACTINIA_USER"] = 'actinia-gdi'
os.environ["ACTINIA_PASSWORD"] = 'actinia-gdi'
os.environ["AUTH"] = 'actinia-gdi:actinia-gdi'
os.environ["ACTINIA_URL"] = 'http://localhost:8088'

ACTINIA_VERSION = 'v3'
ACTINIA_BASEURL = 'http://localhost:8088'
ACTINIA_URL = ACTINIA_BASEURL + "/api/" + ACTINIA_VERSION
ACTINIA_AUTH = HTTPBasicAuth("actinia-gdi", "actinia-gdi")
sys.path.append(os.path.join(gisbase, "etc", "python"))

# Import the GRASS GIS packages we need.
import grass.script as gs
import grass.jupyter as gj

# Start GRASS Session
## Set your grass data location
gj.init("../actinia-core-data/grassdb", "nc_spm_08", "PERMANENT")

## Setup

In [37]:
samples=400

## Baseline - r.info test

The base line test runs each command 100 times

### GRASS (19 ms)

In [38]:

time_log = []
for i in range(samples):
    start_time = time.time()
    !r.info elevation -gre >/dev/null 2>&1
    end_time = time.time()
    run_time = end_time - start_time
    time_log.append(run_time)


print(f"Median: {statistics.median(time_log)}")

Median: 0.19030392169952393


### Actinia

#### No-Cache

In [ ]:
time_log = []
for i in range(samples):
    # !curl -u 'actinia-gdi:actinia-gdi' -X DELETE "http://localhost:8088/api/v3/download_cache" | jq
    start_time = time.time()
    !curl -u 'actinia-gdi:actinia-gdi' -X GET "http://localhost:8088/api/v3/locations/nc_spm_08/mapsets/PERMANENT/raster_layers/elevation" >/dev/null 2>&1
    end_time = time.time()
    run_time = end_time - start_time
    time_log.append(run_time)


print(time_log)
print(f"Median: {statistics.median(time_log)}")



#### Cached (63 ms)

In [36]:
time_log = []
for i in range(samples):
    # !curl -u 'actinia-gdi:actinia-gdi' -X DELETE "http://localhost:8088/api/v3/download_cache" | jq
    start_time = time.time()
    !curl -u 'actinia-gdi:actinia-gdi' -X GET "http://localhost:8088/api/v3/locations/nc_spm_08/mapsets/PERMANENT/raster_layers/elevation">/dev/null 2>&1
    end_time = time.time()
    run_time = end_time - start_time
    time_log.append(run_time)

print(f"Median: {statistics.median(time_log)}")


Median: 0.6288026571273804


### OpenPlains

#### Actinia (cached) OpenPlains No-Cache (66 ms)

In [34]:
time_log = []
for i in range(samples):
    start_time = time.time()
    !curl -X GET "http://localhost:8005/savana/r/locations/nc_spm_08/mapsets/PERMANENT/raster_layers/elevation/no_cache">/dev/null 2>&1
    end_time = time.time()
    run_time = end_time - start_time
    time_log.append(run_time)


# print(time_log)
print(f"Median: {statistics.median(time_log)}")

Median: 0.6593670845031738


#### Cached (21 ms)

In [35]:
time_log = []
for i in range(samples):
    start_time = time.time()
    # Don't print curl response
    !curl -X GET "http://localhost:8005/savana/r/locations/nc_spm_08/mapsets/PERMANENT/raster_layers/elevation" >/dev/null 2>&1
    end_time = time.time()
    run_time = end_time - start_time
    time_log.append(run_time)


# print(time_log)
print(f"Median: {statistics.median(time_log)}")

Median: 0.21401607990264893
